In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read the text file
with open('/content/drive/MyDrive/hackathone/sherlock-holm.es_stories_plain-text_advs.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

In [5]:
input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [6]:
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [7]:
X = input_sequences[:, :-1]
y = input_sequences[:, -1]

In [8]:
y = np.array(tf.keras.utils.to_categorical(y, num_classes=total_words))

In [9]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150))
model.add(Dense(total_words, activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 17, 100)           820000    
                                                                 
 lstm (LSTM)                 (None, 150)               150600    
                                                                 
 dense (Dense)               (None, 8200)              1238200   
                                                                 
Total params: 2,208,800
Trainable params: 2,208,800
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=50, verbose=1)

Epoch 1/50
3010/3010 [==============================] - 259s 85ms/step - loss: 6.2345 - accuracy: 0.0761
Epoch 2/50
3010/3010 [==============================] - 255s 85ms/step - loss: 5.5116 - accuracy: 0.1261
Epoch 3/50
3010/3010 [==============================] - 256s 85ms/step - loss: 5.1192 - accuracy: 0.1482
Epoch 4/50
3010/3010 [==============================] - 255s 85ms/step - loss: 4.7844 - accuracy: 0.1655
Epoch 5/50
3010/3010 [==============================] - 252s 84ms/step - loss: 4.4733 - accuracy: 0.1854
Epoch 6/50
3010/3010 [==============================] - 253s 84ms/step - loss: 4.1828 - accuracy: 0.2062
Epoch 7/50
3010/3010 [==============================] - 254s 85ms/step - loss: 3.9042 - accuracy: 0.2326
Epoch 8/50
3010/3010 [==============================] - 252s 84ms/step - loss: 3.6386 - accuracy: 0.2651
Epoch 9/50
3010/3010 [==============================] - 250s 83ms/step - loss: 3.3919 - accuracy: 0.3000
Epoch 10/50
3010/3010 [==============================] 

In [14]:
seed_text = "I will leave if they"
next_words = 10

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)

1/1 [==============================] - 0s 25ms/step
I will leave if they were identical it would have been better to expect it
